# 🥇 Gold Layer — Business Metrics

**Annie's Magic Numbers Medallion Architecture**

This notebook produces analytical tables for Power BI using clean Silver data.

### 🔐 Configuration — ADLS Gen2 Authentication

In [ ]:
spark.conf.set(
    "fs.azure.account.key.anniedatalake123.dfs.core.windows.net",
    "<PASTE_STORAGE_ACCOUNT_KEY_1_HERE>"
)

### 🟦 Path Setup & Imports

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

container_name = "annie-data"
storage_account = "anniedatalake123"

base_path = f"abfss://{container_name}@{storage_account}.dfs.core.windows.net/"
silver_path = base_path + "silver/"
gold_path = base_path + "gold/"

### 🟦 Register Silver Tables as Temporary Views

In [ ]:
spark.read.format("delta").load(silver_path + "sales").createOrReplaceTempView("sv_sales")
spark.read.format("delta").load(silver_path + "purchases").createOrReplaceTempView("sv_purchases")
spark.read.format("delta").load(silver_path + "beg_inventory").createOrReplaceTempView("sv_beg_inv")
spark.read.format("delta").load(silver_path + "end_inventory").createOrReplaceTempView("sv_end_inv")

print("✅  Temporary views registered from Silver paths.")

### 🟦 Helper Writer Function

In [ ]:
def write_gold(df, table_name, partition_by=None):
    writer = (
        df.write
          .format("delta")
          .mode("overwrite")
          .option("overwriteSchema", "true")
    )
    if partition_by:
        writer = writer.partitionBy(partition_by)
    
    target_path = gold_path + table_name
    writer.save(target_path)
    
    count = spark.read.format("delta").load(target_path).count()
    print(f"   ✅  gold.{table_name} saved to {target_path}  →  {count:,} rows")

### 🥇 Gold — Core Cost Lookup

In [ ]:
cost_lookup = spark.sql("""
    SELECT
        brand,
        description,
        PERCENTILE_APPROX(cost_per_unit, 0.5)  AS median_cost_per_unit,
        AVG(cost_per_unit)                       AS avg_cost_per_unit
    FROM sv_purchases
    WHERE cost_per_unit IS NOT NULL AND cost_per_unit > 0
    GROUP BY brand, description
""")
cost_lookup.createOrReplaceTempView("cost_lookup")

### 🥇 Gold — Sales Enriched

In [ ]:
sales_enriched = spark.sql("""
    SELECT
        s.*,
        COALESCE(c.median_cost_per_unit, s.sales_price * 0.60) AS cost_per_unit,
        ROUND(s.sales_dollars - (COALESCE(c.median_cost_per_unit, s.sales_price * 0.60) * s.sales_quantity), 2) AS profit_dollars,
        ROUND(CASE WHEN s.sales_dollars = 0 THEN NULL 
              ELSE ((s.sales_dollars - COALESCE(c.median_cost_per_unit, s.sales_price * 0.60) * s.sales_quantity) / s.sales_dollars) * 100 
              END, 2) AS margin_pct
    FROM sv_sales s
    LEFT JOIN cost_lookup c ON s.brand = c.brand AND s.description = c.description
""")
sales_enriched.createOrReplaceTempView("gold_se")
write_gold(sales_enriched, "sales_enriched", partition_by="brand")

### 🥇 Gold — Product & Brand Profitability

In [ ]:
product_profitability = spark.sql("""
    SELECT brand, description, size, classification,
           SUM(sales_quantity) AS total_units_sold,
           ROUND(SUM(sales_dollars), 2) AS total_revenue,
           ROUND(SUM(profit_dollars), 2) AS total_profit_dollars,
           ROUND(AVG(margin_pct), 2) AS avg_margin_pct
    FROM gold_se
    GROUP BY brand, description, size, classification
""")
write_gold(product_profitability, "product_profitability")

brand_profitability = spark.sql("""
    SELECT brand, FIRST(classification) AS classification,
           ROUND(SUM(sales_dollars), 2) AS total_revenue,
           ROUND(SUM(profit_dollars), 2) AS total_profit_dollars,
           ROUND(AVG(margin_pct), 2) AS avg_margin_pct
    FROM gold_se
    GROUP BY brand
""")
write_gold(brand_profitability, "brand_profitability")